In [37]:
import scipy.sparse.csgraph as csg
import pprint
import pandas as pd
import numpy as np
from itertools import permutations
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [17]:
transitions_df = pd.read_csv('transitions_table.csv')
movements_df = pd.read_csv('movements_table.csv')

In [20]:
countries = list(transitions_df['from'].unique())

In [21]:
country_idx = {country: i for i, country in enumerate(countries)}

print('Correspondance')
pprint.PrettyPrinter(width=30).pprint(list(enumerate(countries)))

Correspondance
[(0, 'Санкт-Петербург'),
 (1, 'Рига'),
 (2, 'Москва'),
 (3, 'Стокгольм'),
 (4, 'Хельсінкі'),
 (5, 'Осло'),
 (6, 'Франкфурт'),
 (7, 'Прага'),
 (8, 'Берлін'),
 (9, 'Гамбург'),
 (10, 'Копенгаген'),
 (11, 'Гданськ'),
 (12, 'Лондон'),
 (13, 'Дублін'),
 (14, 'Мадрид'),
 (15, 'Порту'),
 (16, 'Нант'),
 (17, 'Лісабон'),
 (18, 'Стамбул'),
 (19, 'Афіни'),
 (20, 'Софія'),
 (21, 'Бухарест'),
 (22, 'Анкара'),
 (23, 'Палермо'),
 (24, 'Венеція'),
 (25, 'Белград'),
 (26, 'Мюнхен'),
 (27, 'Будапешт'),
 (28, 'Берн'),
 (29, 'Відень'),
 (30, 'Київ'),
 (31, 'Мінськ'),
 (32, 'Львів'),
 (33, 'Варшава'),
 (34, 'Севастополь'),
 (35, 'Кишинів'),
 (36, 'Брюссель'),
 (37, 'Барселона'),
 (38, 'Марсель'),
 (39, 'Амстердам'),
 (40, 'Париж'),
 (41, 'Рим')]


In [22]:
for col in ['from', 'to']:
    transitions_df[col] = transitions_df[col].map(lambda x: country_idx[x])
    movements_df[col] = movements_df[col].map(lambda x: country_idx[x])

In [23]:
display(transitions_df[:5])
display(movements_df[:5])

,from,to
0,0,6
1,1,13
2,2,41
3,3,21
4,4,33


,from,to,steps
0,5,3,3
1,5,10,4
2,3,10,4
3,3,11,4
4,3,4,3


In [24]:
transitions_df = transitions_df.to_numpy()
movements_df = movements_df.to_numpy()

In [26]:
N = len(countries)
g_dense = np.zeros((N, N))

In [27]:
for k in range(movements_df.shape[0]):
    k, j, steps = movements_df[k]
    g_dense[k, j] = steps
    g_dense[j, k] = steps

for k in range(transitions_df.shape[0]):
    k, j = transitions_df[k]
    g_dense[k, j] = 1

In [29]:
g_dense[:2]

array([[0., 4., 4., 0., 3., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [4., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 4., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [30]:
g_sparse = csg.csgraph_from_dense(g_dense, null_value=0)

In [32]:
g_sparse.data[:5]

array([4., 4., 3., 1., 4.])

In [33]:
dist_matrix, preds = csg.floyd_warshall(g_sparse, directed=True, return_predecessors=True)

In [35]:
dist_matrix[:2]

array([[0., 4., 4., 6., 3., 8., 1., 4., 6., 4., 6., 6., 7., 5., 2., 4.,
        8., 5., 6., 8., 6., 7., 5., 7., 5., 6., 3., 8., 5., 6., 7., 6.,
        7., 4., 7., 4., 4., 5., 5., 3., 6., 5.],
       [4., 0., 6., 6., 3., 5., 5., 5., 2., 4., 6., 4., 4., 1., 6., 7.,
        6., 8., 9., 6., 6., 7., 8., 5., 5., 8., 6., 6., 8., 4., 6., 3.,
        7., 4., 5., 8., 7., 3., 5., 7., 7., 7.]])

In [36]:
mustpass = ['Франкфурт', 'Рига', 'Лондон', 'Софія', 'Київ', 'Марсель']
mustpass_idx = [country_idx[c] for c in mustpass]

In [38]:
K = len(mustpass_idx)
min_dist = np.inf
min_path = None
for perm in tqdm(permutations(mustpass_idx)):
    dist = 0
    for k in range(K-1):
        i, j = perm[k:k+2]
        dist += dist_matrix[i, j]
    if dist < min_dist:
        min_dist = dist
        min_path = perm

720it [00:00, 286599.50it/s]


In [41]:
def reconstruct_path(partial_path):
    path = []
    for i in range(len(partial_path) - 1):
        u, v = partial_path[i], partial_path[i+1]
        segm = []
        while u != v:
            v = preds[u, v]
            segm.append(v)
        segm.reverse()
        path = path + segm
    path.append(partial_path[-1])
    return path

In [42]:
rec_path = reconstruct_path(min_path)

In [43]:
rec_path_str = [countries[i] for i in rec_path]

print(min_dist)
print(rec_path_str)

15.0
['Київ', 'Санкт-Петербург', 'Франкфурт', 'Мюнхен', 'Берн', 'Стамбул', 'Рига', 'Хельсінкі', 'Варшава', 'Марсель', 'Софія', 'Лондон']
